# Tutorial: Hierarchical Recurrent Connectivity

This notebook shows how to create hierarchical connectivity patterns and use them in network architectures.

**Key concepts:**
- Building multi-tier connectivity patterns with nested blocks using `structure.hierarchical_blocks()`
- Customizing connectivity density at each tier with `tier_fractions`
- Visualizing hierarchical network structure with automatic block detection
- Understanding hierarchical mask properties (sparsity, fan-out, etc.)


In [ ]:
# Imports
import os
import tempfile

import matplotlib.pyplot as plt
import numpy as np

from soen_toolkit.core.utils.hierarchical_mask import analyze_mask, create_hierarchical_mask
from soen_toolkit.nn import Graph, init, layers, structure

## Create Hierarchical Mask

Creates a 3-tier hierarchy with base_size=4:
- Tier 0: blocks of 4 nodes (100% connected)
- Tier 1: blocks of 16 nodes (10% selected, fully connected)
- Tier 2: blocks of 64 nodes (10% selected, fully connected)

Total: 64 nodes with symmetric connections.


In [ ]:
# Create mask (uses default tier_fractions=[1.0, 0.5, 0.25])
mask = create_hierarchical_mask(levels=3, base_size=4, tier_fractions=[1.0, 0.1, 0.1])
stats = analyze_mask(mask)


# Visualize mask
fig, ax = plt.subplots(figsize=(7, 6))
ax.imshow(mask, cmap="binary", aspect="auto")
ax.set_title(f"Hierarchical Mask ({mask.shape[0]}×{mask.shape[1]})")
ax.set_xlabel("Source")
ax.set_ylabel("Destination")
plt.tight_layout()
plt.show()

In [ ]:
## Build a Model with Custom Hierarchical Masks

# Create two different hierarchical masks
mask1 = create_hierarchical_mask(levels=3, base_size=4)  # 64 nodes
mask2 = create_hierarchical_mask(levels=2, base_size=4)  # 16 nodes

# Use a real temporary directory for mask files
with tempfile.TemporaryDirectory() as temp_dir:
    mask1_file = os.path.join(temp_dir, "mask1.npz")
    mask2_file = os.path.join(temp_dir, "mask2.npz")
    np.savez(mask1_file, mask=mask1)
    np.savez(mask2_file, mask=mask2)

    # Build model with different hierarchies per layer
    g_mixed = Graph(dt=37, network_evaluation_method="layerwise")

    g_mixed.add_layer(0, layers.MultiplierWICC(dim=64))  # WICC (With Collection Coil) multiplier
    g_mixed.connect(
        0,
        0,
        structure=structure.custom(
            mask1_file,
            # Store metadata for visualization
            visualization_metadata={"hierarchical": {"levels": 3, "base_size": 4}},
        ),
        init=init.constant(0.1),
    )

    g_mixed.add_layer(1, layers.MultiplierWICC(dim=16))  # WICC multiplier
    g_mixed.connect(1, 1, structure=structure.custom(mask2_file, visualization_metadata={"hierarchical": {"levels": 2, "base_size": 4}}), init=init.constant(0.1))

    # Connect layers with sparse connectivity
    g_mixed.connect(0, 1, structure=structure.sparse(sparsity=0.1), init=init.xavier_uniform())

    kpis = g_mixed.compute_summary()["kpis"]

    # Visualize the hierarchical structure
    # Each layer uses its stored metadata to show hierarchical blocks
    g_mixed.visualize_grid_of_grids()